# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
DATA_FOLDER = "/home/vinz/Desktop/ADA/ADA2017-Tutorials/02 - Intro to Pandas/Data" # Use the data folder provided in Tutorial 02 - Intro to Pandas.
#DATA_FOLDER = "./Data"

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average* per year of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [3]:
# Setup for location of the dataset of the task 2
MICROBIOME_FOLDER = DATA_FOLDER + "/microbiome"

**PROOFREADING** What do you mean by the questions below? keep data from different MID into different columns?

In [4]:
# Write your answer here
# TODO: Should the data from the spreadsheets be merged in different columns instead of adding indexes as we are doing now
#        Should at least the tissue and stool from the same group of patient be merged?


### Basic analysis of the file formatting:

***
**For the files MIDn.xls with n in [1,9]**

**column 1:**

We see that the first column in the files contain the scientific classification of the microbiomes
Altough it could be kept as a single string, it would have more meaning if splitted
The scientific classification contains the following subdivisions (https://en.wikipedia.org/wiki/Taxonomic_rank): 

       Domain, Kingdom, Phylum, Class, Order, Family, Genus, Species
       
 With this classification, we have a problem: only 6 strings are given in the data set, whilst we have 8 potential divisions in the classification.
 
 After analysis, it seems that Kingdom is not used with bacterias (https://en.wikipedia.org/wiki/Bacteria)
 and always the same as the Phylum in Archea(https://en.wikipedia.org/wiki/Archaea). Therefore, **we will not use Kingdom as a division**.
 
 Also, Species only has a sense in the Eucaryote Domain, for which we have no data, so **we will not use Species as a division**.
 
 In addition, after playing with the data, we found cases where the Family can be named: "Incertae Sedis", which would be classified as the family and genus in our classification. Therefore, we need to check for those cases and re-concatenate the strings to return a proper list in every cases.

**PROOFREADING**: Don't understand "After analysis, it seems that Kingdom is not used with bacterias (https://en.wikipedia.org/wiki/Bacteria) and always the same as the Phylum in Archea(https://en.wikipedia.org/wiki/Archaea). "

In [5]:
# Lets keep those names in a list for further use
scientific_classification = ["domain", "phylum", "class", "order", "family", "genus"]

**Column 2:**

We see that the second column contains an integer value. We can suppose that this value is the number of samples
containing the genus defined by the first column. There's no title to the column, so we don't know yet where
those values came from.

***

**For the file "Metadata.xls"**

**Column 1:**

Titled **"BARCODE"**, give xls file identifier for which the two other columns give more information

**Column 2:**

Titled **"GROUP"**, gives the groups from which each dataset has been sampled. Two informations are contained in those groups, the first is given by "NEC", "Control" or "EXTRACTION CONTROL" and the second is the numbering of the group (which is likely a phase of test), either "1" or "2". We will split those informations in two column, because while using the dataset, we might want to combine all the "NEC" patients or all the patient for a specific phase.

**Column 3:**

Titled **"SAMPLE"**, gives the type of sample that was taken, either tissue, stool or NA. Each group had both types of samples taken. 

***

**PROOFREADING** Do you think control and extraction control might be the same thing?

### Desired formating of the data after analysis

The simple analysis above tells us what are the columns that we will want in our Data Frame

1. **6** column for classification, 1 for each classifier in the "scientific_classification" list. This will caracterise each microbiome individually.

2. **1** column will contain the value associated with each microbiome measurement.

3. **2** columns describing the group of the sample from the metadata. The first column will be called **Group Type** and will contain the "NEC", "Control" or "EXTRACTION CONTROL" value. The second column will be called **Group Phase** and will contain either "1", "2" or "unknow", "unknow" will be used for the "EXCTRACTON CONTROL" group.

4. **1** column describing the type of sample taken from the metadata. This column will be called **Sample** This will be either "tissue", "stool" or "unknow", "unknow" in the case of the EXTRACTION CONTROL group.

For a total of **10** columns


In [6]:
metadata_list = ["group_type", "group_phase", "sample"]

In [7]:
df_col_list = scientific_classification + metadata_list + ["value"]

In [8]:
print(len(df_col_list))
print(df_col_list)

10
['domain', 'phylum', 'class', 'order', 'family', 'genus', 'group_type', 'group_phase', 'sample', 'value']


***
### We want to extract the metadata from the metadata.xls file

The metadata is needed before we start extracting the data from the MB files to create the DataFrame with all the desired columns.


In [9]:
#The actual name of the excel sheet is "Sheet1" and not "Sheet 1" as in the other files.
metadata_raw = pd.read_excel(MICROBIOME_FOLDER+"/metadata.xls", sheetname='Sheet1', header=0)
metadata_raw.columns = metadata_raw.columns.str.lower()

In [10]:
## Extract group phase and group type
groups = metadata_raw["group"]
group_type = []
group_phase = []
for group in groups:
    # A special case for the extraction control, we don't want to split it
    if group == "EXTRACTION CONTROL":
        group_type.append(group)
        group_phase.append("")
    else:
        type, phase, *_ = group.split()
        group_type.append(type)
        group_phase.append(phase)

In [11]:
metadata_raw['group_type'] = pd.Series(group_type)
metadata_raw['group_phase'] = pd.Series(group_phase)
metadata = metadata_raw.drop("group",  axis=1)
metadata

,barcode,sample,group_type,group_phase
0,MID1,NaN,EXTRACTION CONTROL,
1,MID2,tissue,NEC,1
2,MID3,tissue,Control,1
3,MID4,tissue,NEC,2
4,MID5,tissue,Control,2
5,MID6,stool,NEC,1
6,MID7,stool,Control,1
7,MID8,stool,NEC,2
8,MID9,stool,Control,2


In [12]:
# Make sure SAMPLE doesn't contain NaN, replace it by an empty string
# This will give use prettier column names later on
metadata["sample"].fillna(value="", inplace=True)
metadata

,barcode,sample,group_type,group_phase
0,MID1,,EXTRACTION CONTROL,
1,MID2,tissue,NEC,1
2,MID3,tissue,Control,1
3,MID4,tissue,NEC,2
4,MID5,tissue,Control,2
5,MID6,stool,NEC,1
6,MID7,stool,Control,1
7,MID8,stool,NEC,2
8,MID9,stool,Control,2


The format of the metadata is now as we want it

***
### We now want to extract the data from the datasheets

** We create a function to extract the classifications **

We want to extract the classifications from the first column of the datasheets. We will need to do string parsing. Regex are a great tool to remove the unwanted characters from the format. The only unwanted characters are the double quotes ". Also, as said before, we need to manage the case where we have the name "Incertae Sedis" as a Family (4th position)

In [13]:
import re

def get_classifiers(classifier_string):
    """Replaces " in the classifier_string and 
    splits the string to have an indexable list
    """
    classified_list = re.subn("\"", "", classifier_string)[0].split()
    
    # Special case management
    if len(classified_list) > 6:
        # We join the first extra location with the family
        classified_list[4] = ' '.join([classified_list[4], classified_list[5]])
        del classified_list[5]
        
        # Allow for harbitrary number of words in the genus
        classified_list[5] = ' '.join(classified_list[6:])
        del classified_list[6:]
        
    return classified_list

** We can now attempt to create the desired dataframe **

In [14]:
# In this section, we test the analysis of a single datasheet

test_data = pd.read_excel(MICROBIOME_FOLDER+"/MID1.xls", sheetname='Sheet 1', header=None)
test_data.columns = ["raw_classification", "value"]
classifier_df = pd.DataFrame(columns=df_col_list)
classifier_array = [get_classifiers(row.raw_classification) for row in test_data.itertuples()]
classifier_series = pd.DataFrame(classifier_array, columns=scientific_classification)
classifier_series["value"] = test_data["value"]
classifier_series

,domain,phylum,class,order,family,genus,value
0,Archaea,Crenarchaeota,Thermoprotei,Desulfurococcales,Desulfurococcaceae,Ignisphaera,7
1,Archaea,Crenarchaeota,Thermoprotei,Desulfurococcales,Pyrodictiaceae,Pyrolobus,2
2,Archaea,Crenarchaeota,Thermoprotei,Sulfolobales,Sulfolobaceae,Stygiolobus,3
3,Archaea,Crenarchaeota,Thermoprotei,Thermoproteales,Thermofilaceae,Thermofilum,3
4,Archaea,Euryarchaeota,Methanomicrobia,Methanocellales,Methanocellaceae,Methanocella,7
5,Archaea,Euryarchaeota,Methanomicrobia,Methanosarcinales,Methanosarcinaceae,Methanimicrococcus,1
6,Archaea,Euryarchaeota,Methanomicrobia,Methanosarcinales,Methermicoccaceae,Methermicoccus,1
7,Archaea,Euryarchaeota,Archaeoglobi,Archaeoglobales,Archaeoglobaceae,Ferroglobus,1
8,Archaea,Euryarchaeota,Archaeoglobi,Archaeoglobales,Archaeoglobaceae,Geoglobus,1
9,Archaea,Euryarchaeota,Halobacteria,Halobacteriales,Halobacteriaceae,Haloplanus,1


In [15]:
# We reorder the metadata here, this will order the resulting data
metadata = metadata[["barcode", "group_phase", "group_type", "sample"]].sort_values(by=[ 'group_phase', 'group_type', 'sample'])

In [16]:
# Loop over all the datasheets
clean_data = pd.DataFrame(columns=scientific_classification)
for metadata_row in metadata.itertuples():
    raw_data = pd.read_excel(MICROBIOME_FOLDER+"/"+metadata_row.barcode+".xls", sheetname='Sheet 1', header=None)
    # Change column names to something clearer
    raw_data.columns = ["raw_classification", "value"]
    
    # For each datasheet create a local classified set of data
    classifier_array = [get_classifiers(row.raw_classification) for row in raw_data.itertuples()]
    local_classified = pd.DataFrame(classifier_array, columns=scientific_classification)
    
    # Add the columns that are not the classification to the local set of data 
    local_classified[str(metadata_row.barcode)]  = test_data["value"]
    
    # Add the local data to the clean DataFrame
    clean_data = pd.merge(clean_data, local_classified, how="outer", on=scientific_classification)

clean_data.head(10)

,domain,phylum,class,order,family,genus,MID1,MID7,MID3,MID6,MID2,MID9,MID5,MID8,MID4
0,Archaea,Crenarchaeota,Thermoprotei,Desulfurococcales,Desulfurococcaceae,Ignisphaera,7.0,2.0,3.0,7.0,3.0,2.0,3.0,NaN,7.0
1,Archaea,Crenarchaeota,Thermoprotei,Desulfurococcales,Pyrodictiaceae,Pyrolobus,2.0,1.0,NaN,3.0,1.0,NaN,1.0,NaN,NaN
2,Archaea,Crenarchaeota,Thermoprotei,Sulfolobales,Sulfolobaceae,Stygiolobus,3.0,1.0,1.0,7.0,1.0,7.0,1.0,7.0,NaN
3,Archaea,Crenarchaeota,Thermoprotei,Thermoproteales,Thermofilaceae,Thermofilum,3.0,1.0,1.0,1.0,1.0,1.0,4.0,NaN,NaN
4,Archaea,Euryarchaeota,Methanomicrobia,Methanocellales,Methanocellaceae,Methanocella,7.0,1.0,2.0,1.0,2.0,1.0,4.0,NaN,3.0
5,Archaea,Euryarchaeota,Methanomicrobia,Methanosarcinales,Methanosarcinaceae,Methanimicrococcus,1.0,2.0,12.0,4.0,1.0,2.0,2.0,NaN,NaN
6,Archaea,Euryarchaeota,Methanomicrobia,Methanosarcinales,Methermicoccaceae,Methermicoccus,1.0,4.0,2.0,2.0,12.0,4.0,1.0,NaN,NaN
7,Archaea,Euryarchaeota,Archaeoglobi,Archaeoglobales,Archaeoglobaceae,Ferroglobus,1.0,1.0,NaN,4.0,2.0,1.0,1.0,NaN,3.0
8,Archaea,Euryarchaeota,Archaeoglobi,Archaeoglobales,Archaeoglobaceae,Geoglobus,1.0,12.0,NaN,NaN,NaN,12.0,1.0,NaN,NaN
9,Archaea,Euryarchaeota,Halobacteria,Halobacteriales,Halobacteriaceae,Haloplanus,1.0,2.0,1.0,1.0,1.0,2.0,2.0,NaN,NaN


** Lets find all rows that contains only NaN values **

Those rows are not usefull for the analysis and we will drop them.

In [17]:
# Get a table with True in place of a row where all the MID values are null
table_of_null_row = pd.isnull(clean_data[metadata.barcode]).all(axis=1)

# Get the associated indexes
index_of_null_row = table_of_null_row[table_of_null_row].index[:]

# Lets check that the value are actually null
clean_data.iloc[index_of_null_row]

,domain,phylum,class,order,family,genus,MID1,MID7,MID3,MID6,MID2,MID9,MID5,MID8,MID4
419,Bacteria,Proteobacteria,Betaproteobacteria,Burkholderiales,Comamonadaceae,Rhodoferax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
420,Bacteria,Proteobacteria,Betaproteobacteria,Burkholderiales,Comamonadaceae,Simplicispira,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421,Bacteria,Proteobacteria,Betaproteobacteria,Burkholderiales,Comamonadaceae,Tepidicella,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
422,Bacteria,Proteobacteria,Betaproteobacteria,Burkholderiales,Oxalobacteraceae,Undibacterium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
423,Bacteria,Proteobacteria,Betaproteobacteria,Hydrogenophilales,Hydrogenophilaceae,Tepidiphilus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
425,Bacteria,Proteobacteria,Betaproteobacteria,Methylophilales,Methylophilaceae,Methylovorus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
426,Bacteria,Proteobacteria,Betaproteobacteria,Neisseriales,Neisseriaceae,Formivibrio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
427,Bacteria,Proteobacteria,Betaproteobacteria,Neisseriales,Neisseriaceae,Leeia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
428,Bacteria,Proteobacteria,Betaproteobacteria,Neisseriales,Neisseriaceae,Microvirgula,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
430,Bacteria,Proteobacteria,Betaproteobacteria,Neisseriales,Neisseriaceae,Stenoxybacter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
clean_data = clean_data.drop(index_of_null_row)

** Now lets replace all the None values by unknow **

**PROOFREADING** Does below code work ?

In [19]:
clean_data.fillna(value="unknown", inplace=True)
#for item in clean_data.iteritems():
#    print(item)
#    item[1].fillna(value="unknown", inplace=True)

In [20]:
clean_data

,domain,phylum,class,order,family,genus,MID1,MID7,MID3,MID6,MID2,MID9,MID5,MID8,MID4
0,Archaea,Crenarchaeota,Thermoprotei,Desulfurococcales,Desulfurococcaceae,Ignisphaera,7,2,3,7,3,2,3,unknown,7
1,Archaea,Crenarchaeota,Thermoprotei,Desulfurococcales,Pyrodictiaceae,Pyrolobus,2,1,unknown,3,1,unknown,1,unknown,unknown
2,Archaea,Crenarchaeota,Thermoprotei,Sulfolobales,Sulfolobaceae,Stygiolobus,3,1,1,7,1,7,1,7,unknown
3,Archaea,Crenarchaeota,Thermoprotei,Thermoproteales,Thermofilaceae,Thermofilum,3,1,1,1,1,1,4,unknown,unknown
4,Archaea,Euryarchaeota,Methanomicrobia,Methanocellales,Methanocellaceae,Methanocella,7,1,2,1,2,1,4,unknown,3
5,Archaea,Euryarchaeota,Methanomicrobia,Methanosarcinales,Methanosarcinaceae,Methanimicrococcus,1,2,12,4,1,2,2,unknown,unknown
6,Archaea,Euryarchaeota,Methanomicrobia,Methanosarcinales,Methermicoccaceae,Methermicoccus,1,4,2,2,12,4,1,unknown,unknown
7,Archaea,Euryarchaeota,Archaeoglobi,Archaeoglobales,Archaeoglobaceae,Ferroglobus,1,1,unknown,4,2,1,1,unknown,3
8,Archaea,Euryarchaeota,Archaeoglobi,Archaeoglobales,Archaeoglobaceae,Geoglobus,1,12,unknown,unknown,unknown,12,1,unknown,unknown
9,Archaea,Euryarchaeota,Halobacteria,Halobacteriales,Halobacteriaceae,Haloplanus,1,2,1,1,1,2,2,unknown,unknown


We see that some genus are unknown... This seems a bit odd, so we will search if one other of the term is the actual genus

In [21]:
unknown_genus = clean_data[clean_data.genus == 'unknown']
unknown_genus

,domain,phylum,class,order,family,genus,MID1,MID7,MID3,MID6,MID2,MID9,MID5,MID8,MID4
148,Bacteria,Proteobacteria,Alphaproteobacteria,Alphaproteobacteria_incertae_sedis,Elioraea,unknown,1,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown
223,Bacteria,Proteobacteria,Gammaproteobacteria,Gammaproteobacteria_incertae_sedis,Gilvimarinus,unknown,1,unknown,unknown,1,unknown,2,unknown,unknown,unknown
224,Bacteria,Proteobacteria,Gammaproteobacteria,Gammaproteobacteria_incertae_sedis,Solimonas,unknown,2,unknown,unknown,unknown,unknown,162,unknown,unknown,unknown
269,Bacteria,Cyanobacteria,Cyanobacteria,Chloroplast,Bangiophyceae,unknown,2,unknown,unknown,unknown,unknown,unknown,unknown,196,unknown
270,Bacteria,Cyanobacteria,Cyanobacteria,Chloroplast,Chlorarachniophyceae,unknown,85,unknown,unknown,unknown,unknown,unknown,unknown,1,1
271,Bacteria,Cyanobacteria,Cyanobacteria,Chloroplast,Streptophyta,unknown,1388,unknown,unknown,unknown,unknown,unknown,unknown,unknown,1
281,Bacteria,Actinobacteria,Actinobacteria,Acidimicrobidae_incertae_sedis,Ilumatobacter,unknown,unknown,3,2,unknown,unknown,unknown,1,unknown,unknown
384,Bacteria,Proteobacteria,Alphaproteobacteria,Alphaproteobacteria_incertae_sedis,Geminicoccus,unknown,unknown,1,unknown,unknown,unknown,unknown,unknown,unknown,unknown
455,Bacteria,Proteobacteria,Gammaproteobacteria,Gammaproteobacteria_incertae_sedis,Sedimenticola,unknown,unknown,unknown,unknown,unknown,unknown,9,unknown,unknown,unknown
512,Bacteria,Bacteroidetes,Bacteroidetes_incertae_sedis,Marinifilum,unknown,unknown,unknown,unknown,1,unknown,unknown,unknown,unknown,7,12


From the list above, we can see that we have many cases where we have a string containing "incertae_sedis". Normally, this nomenclature is used when a genus doesn't have clear parents (https://en.wikipedia.org/wiki/Incertae_sedis). Because of this we will attempt to use the first class after the incertae_sedis as the name of the genus and keep the other values as unknown.

In the case of "(some term)_genera_incertae_sedis", it means that the term is the genus and the rest is unknown, so we will do a special parse for those cases.

In cases where we don't have incertae sedis, we will avoid doing manipulations. We don't want to induce errors in the dataset because we managed poorly the nomenclature for the specefic cases. In a real situation, we would either need to

In [32]:
def get_incertae_genus(class_, order):
    """Extracts genus from either class or order if either of them contain incertae_sedis"""
    for target in [class_, order]:
        genus, *rest = target.split('_incertae_sedis')
        if len(rest) > 0:
            return genus
    return 'unknown'

In [ ]:
genuses = [get_incertae_genus(c, o) for *_, c, o in unknown_genus[['class', 'order']].itertuples()]

In [24]:
# Insert newly retrieved genus names
clean_data.loc[clean_data.genus == 'unknown', 'genus'] = genuses

In [25]:
clean_data[clean_data.genus == 'unknown']

,domain,phylum,class,order,family,genus,MID1,MID7,MID3,MID6,MID2,MID9,MID5,MID8,MID4
269,Bacteria,Cyanobacteria,Cyanobacteria,Chloroplast,Bangiophyceae,unknown,2,unknown,unknown,unknown,unknown,unknown,unknown,196,unknown
270,Bacteria,Cyanobacteria,Cyanobacteria,Chloroplast,Chlorarachniophyceae,unknown,85,unknown,unknown,unknown,unknown,unknown,unknown,1,1
271,Bacteria,Cyanobacteria,Cyanobacteria,Chloroplast,Streptophyta,unknown,1388,unknown,unknown,unknown,unknown,unknown,unknown,unknown,1
677,Bacteria,Chloroflexi,Dehalococcoidetes,Dehalogenimonas,unknown,unknown,unknown,unknown,unknown,unknown,unknown,6,1,unknown,unknown
712,Bacteria,Acidobacteria,Acidobacteria_Gp16,Gp16,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,17,unknown,unknown


**PROOFREADING** How to replace all order/classes ending in _incertae_sedis with unknown? 
`clean_data.class.str.replace` but I do not know regexs.

Also what do you mean by the question below?

In [ ]:
# We can check how deep the unknow propagates ????

** Now lets manage the index of the DataFrame **

For now, we consider that every column but the value can be considered as a metadata. 

In [27]:
indexed_data = clean_data.set_index(list(scientific_classification))
indexed_data.head(5)

MID1  \
domain  phylum        class           order             family             genus               
Archaea Crenarchaeota Thermoprotei    Desulfurococcales Desulfurococcaceae Ignisphaera     7   
                                                        Pyrodictiaceae     Pyrolobus       2   
                                      Sulfolobales      Sulfolobaceae      Stygiolobus     3   
                                      Thermoproteales   Thermofilaceae     Thermofilum     3   
        Euryarchaeota Methanomicrobia Methanocellales   Methanocellaceae   Methanocella    7   

                                                                                        MID7  \
domain  phylum        class           order             family             genus               
Archaea Crenarchaeota Thermoprotei    Desulfurococcales Desulfurococcaceae Ignisphaera     2   
                                                        Pyrodictiaceae     Pyrolobus       1   
                                      Sulfolobales      Sulfolobaceae      Stygiolobus     1   
                                      Thermoproteales   Thermofilaceae     Thermofilum     1   
        Euryarchaeota Methanomicrobia Methanocellales   Methanocellaceae   Methanocella    1   

                                                                                            MID3  \
domain  phylum        class           order             family             genus                   
Archaea Crenarchaeota Thermoprotei    Desulfurococcales Desulfurococcaceae Ignisphaera         3   
                                                        Pyrodictiaceae     Pyrolobus     unknown   
                                      Sulfolobales      Sulfolobaceae      Stygiolobus         1   
                                      Thermoproteales   Thermofilaceae     Thermofilum         1   
        Euryarchaeota Methanomicrobia Methanocellales   Methanocellaceae   Methanocella        2   

                                                                                        MID6  \
domain  phylum        class           order             family             genus               
Archaea Crenarchaeota Thermoprotei    Desulfurococcales Desulfurococcaceae Ignisphaera     7   
                                                        Pyrodictiaceae     Pyrolobus       3   
                                      Sulfolobales      Sulfolobaceae      Stygiolobus     7   
                                      Thermoproteales   Thermofilaceae     Thermofilum     1   
        Euryarchaeota Methanomicrobia Methanocellales   Methanocellaceae   Methanocella    1   

                                                                                        MID2  \
domain  phylum        class           order             family             genus               
Archaea Crenarchaeota Thermoprotei    Desulfurococcales Desulfurococcaceae Ignisphaera     3   
                                                        Pyrodictiaceae     Pyrolobus       1   
                                      Sulfolobales      Sulfolobaceae      Stygiolobus     1   
                                      Thermoproteales   Thermofilaceae     Thermofilum     1   
        Euryarchaeota Methanomicrobia Methanocellales   Methanocellaceae   Methanocella    2   

                                                                                            MID9  \
domain  phylum        class           order             family             genus                   
Archaea Crenarchaeota Thermoprotei    Desulfurococcales Desulfurococcaceae Ignisphaera         2   
                                                        Pyrodictiaceae     Pyrolobus     unknown   
                                      Sulfolobales      Sulfolobaceae      Stygiolobus         7   
                                      Thermoproteales   Thermofilaceae     Thermofilum         1   
        Euryarchaeota Methanomicrobia Methanocellales   Methanocellaceae   Methanocella        

In [28]:
indexed_data.index.is_unique

True

** Now lets give the column to more meaningfull names **

The ordering of the columns is already managed to give a nice output. This is why we reordered the metadata earlier on.

In [30]:
pretty_data = pd.DataFrame(data=indexed_data.values,
                           index=indexed_data.index,
                           columns=[metadata.group_phase.get_values(),
                                    metadata.group_type.get_values(),
                                    metadata["sample"].get_values()])
pretty_data.head(5)

\
                                                                                        EXTRACTION CONTROL   
                                                                                                             
domain  phylum        class           order             family             genus                             
Archaea Crenarchaeota Thermoprotei    Desulfurococcales Desulfurococcaceae Ignisphaera                   7   
                                                        Pyrodictiaceae     Pyrolobus                     2   
                                      Sulfolobales      Sulfolobaceae      Stygiolobus                   3   
                                      Thermoproteales   Thermofilaceae     Thermofilum                   3   
        Euryarchaeota Methanomicrobia Methanocellales   Methanocellaceae   Methanocella                  7   

                                                                                              1  \
                                                                                        Control   
                                                                                          stool   
domain  phylum        class           order             family             genus                  
Archaea Crenarchaeota Thermoprotei    Desulfurococcales Desulfurococcaceae Ignisphaera        2   
                                                        Pyrodictiaceae     Pyrolobus          1   
                                      Sulfolobales      Sulfolobaceae      Stygiolobus        1   
                                      Thermoproteales   Thermofilaceae     Thermofilum        1   
        Euryarchaeota Methanomicrobia Methanocellales   Methanocellaceae   Methanocella       1   

                                                                                                  \
                                                                                                   
                                                                                          tissue   
domain  phylum        class           order             family             genus                   
Archaea Crenarchaeota Thermoprotei    Desulfurococcales Desulfurococcaceae Ignisphaera         3   
                                                        Pyrodictiaceae     Pyrolobus     unknown   
                                      Sulfolobales      Sulfolobaceae      Stygiolobus         1   
                                      Thermoproteales   Thermofilaceae     Thermofilum         1   
        Euryarchaeota Methanomicrobia Methanocellales   Methanocellaceae   Methanocella        2   

                                                                                               \
                                                                                          NEC   
                                                                                        stool   
domain  phylum        class           order             family             genus                
Archaea Crenarchaeota Thermoprotei    Desulfurococcales Desulfurococcaceae Ignisphaera      7   
                                                        Pyrodictiaceae     Pyrolobus        3   
                                      Sulfolobales      Sulfolobaceae      Stygiolobus      7   
                                      Thermoproteales   Thermofilaceae     Thermofilum      1   
        Euryarchaeota Methanomicrobia Methanocellales   Methanocellaceae   Methanocella     1   

                                                                                                \
                                                                                                 
                                                                                        tissue   
domain  phylum        class           order             family             genus                 
Archaea Crenarchaeota Thermoprotei    Desulfurococcales De

In [31]:
#Lets give a name to the columns
pretty_data.columns.names = ["Group Number", "Group Type", "Sample"]
pretty_data.head(1000)

Group Number                                                                                                                                     \
Group Type                                                                                                                   EXTRACTION CONTROL   
Sample                                                                                                                                            
domain   phylum         class                        order               family                           genus                                   
Archaea  Crenarchaeota  Thermoprotei                 Desulfurococcales   Desulfurococcaceae               Ignisphaera                         7   
                                                                         Pyrodictiaceae                   Pyrolobus                           2   
                                                     Sulfolobales        Sulfolobaceae                    Stygiolobus                         3   
                                                     Thermoproteales     Thermofilaceae                   Thermofilum                         3   
         Euryarchaeota  Methanomicrobia              Methanocellales     Methanocellaceae                 Methanocella                        7   
                                                     Methanosarcinales   Methanosarcinaceae               Methanimicrococcus                  1   
                                                                         Methermicoccaceae                Methermicoccus                      1   
                        Archaeoglobi                 Archaeoglobales     Archaeoglobaceae                 Ferroglobus                         1   
                                                                                                          Geoglobus                           1   
                        Halobacteria                 Halobacteriales     Halobacteriaceae                 Haloplanus                          1   
                                                                                                          Haloquadratum                       4   
                                                                                                          Halosimplex                         1   
                                                                                                          Natronobacterium                    2   
                                                                                                          Natronomonas                        4   
                        Methanococci                 Methanococcales     Methanocaldococcaceae            Methanotorris                       1   
                        Methanopyri                  Methanopyrales      Methanopyraceae                  Methanopyrus                       12   
                        Thermoplasmata               Thermoplasmatales   Thermoplasmatales_incertae_sedis Thermogymnomonas                    2   
Bacteria Actinobacteria Actinobacteria               Actinomycetales     Acidothermaceae                  Acidothermus                        2   
                                                                         Beutenbergiaceae                 Salana                              1   
                                                                         Bogoriellaceae                   Bogoriella                          1   
                                                                         Cellulomonadaceae                Tropheryma                          1   
                                                                         Corynebacteriaceae               Turicella                           2   
                                                                         Cryptosporangiaceae              Cryptosporangium                   53   
                               

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
#HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here